In [24]:
import tensorflow as tf
from tensorflow import keras


In [25]:
epochs = 50
image_size = (64, 64) # h x w
data_dir = r'../input/animal-faces/afhq/train/'

dataset_train = keras.preprocessing.image_dataset_from_directory(
    data_dir, label_mode='categorical',
    image_size=image_size, batch_size=32, seed=42, validation_split=0.05, subset="training"
)

dataset_validation = keras.preprocessing.image_dataset_from_directory(
    data_dir, label_mode='categorical',
    image_size=image_size, batch_size=32, seed=42, validation_split=0.05, subset="validation"
)

dataset_train = dataset_train.map(lambda x, y: (((x - 127.5) / 127.5), y)) # Normalizing to -1,1
dataset_validation = dataset_validation.map(lambda x, y: (((x - 127.5) / 127.5), y)) # Normalizing to -1,1

In [26]:
data_dir = r'../input/animal-faces/afhq/val/'

dataset_test = keras.preprocessing.image_dataset_from_directory(
    data_dir, label_mode='categorical',
    image_size=image_size, batch_size=32
)

dataset_test = dataset_test.map(lambda x, y: (((x - 127.5) / 127.5), y)) # Normalizing to -1,1


In [27]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])


In [28]:
inception_model = keras.applications.inception_v3.InceptionV3(
    input_shape=(299, 299, 3),
    weights='imagenet',
    include_top=False
)

In [29]:
image_batch, label_batch = next(iter(dataset_train))
print(image_batch.shape, label_batch.shape)
resized = tf.image.resize(image_batch, [299, 299], preserve_aspect_ratio=True, method='nearest')

feature_batch = inception_model(resized)
print(feature_batch.shape)
# (32, 8, 8, 2048) kimenete lesz a képeknek

In [30]:
# Freezing the base model
inception_model.trainable = False

Important note about BatchNormalization layers

Many models contain tf.keras.layers.BatchNormalization layers. This layer is a special case and precautions should be taken in the context of fine-tuning, as shown later in this tutorial.

When you set layer.trainable = False, the BatchNormalization layer will run in inference mode, and will not update its mean and variance statistics.

When you unfreeze a model that contains BatchNormalization layers in order to do fine-tuning, you should keep the BatchNormalization layers in inference mode by passing training = False when calling the base model. Otherwise, the updates applied to the non-trainable weights will destroy what the model has learned.

In [31]:
inception_model.summary()


In [32]:
# Az összerakott modell
inputs = keras.Input(shape=(64, 64, 3))
x = data_augmentation(inputs)
resized = tf.image.resize(x, [299, 299], preserve_aspect_ratio=True, method='nearest')
x = inception_model(resized, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(3)(x)
outputs = keras.layers.Activation("softmax")(x)
model = keras.Model(inputs, outputs)

In [33]:
base_learning_rate = 0.0001
model.compile(optimizer=keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [34]:
model.summary()


In [35]:
len(model.trainable_variables)


In [36]:
initial_epochs = 10

loss0, accuracy0 = model.evaluate(dataset_validation)


In [37]:
print("initial loss: {:.4f}".format(loss0))
print("initial accuracy: {:.4f}".format(accuracy0))

In [38]:
history = model.fit(dataset_train,
                    epochs=initial_epochs,
                    validation_data=dataset_validation)

In [44]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


In [45]:
loss, accuracy = model.evaluate(dataset_test)
print('Test accuracy :', accuracy)


In [46]:
model.save("animalFacesClassifier.h5")